# Data Manipulation
In this notebook, we explore how the composition of the training set for neural networks can lead to biased outcomes. We explore this in a categorization task with two classes by manipulating the balance of training examples across these classes.

Here we will install the relevant library in order for this notebook to run.

In [20]:
%pip install aitk --quiet

Running this next code block will allow us to import the additional libraries for this notebook.

In [21]:
from aitk.utils import gallery, array_to_image
from aitk.networks import SimpleNetwork

import tensorflow
from tensorflow.keras.datasets import mnist

import numpy as np

from random import shuffle

## Manipulating the Data
In this section, we will perform some manipulations on a dataset in order to show how the composition of a dataset can affect the efficacy of a network.

To demonstrate dataset composition's relevance to network efficacy, we will use a part of the MNIST dataset. MNIST is a data set composed of hand-written digits. Specifically, we will use the 4's and 5's in this dataset in order to demonstrate how over or underrepresenting one of these two digits will affect the network's ability to accurately recognize both digits. The reason we chose 4 and 5 is that they have some similarities, such as the horizontal bar through the middle, and some differences, such as the pointed top for the 4 and the flat top for the 5.

We will start with an equal number of 4's and 5's, and then we will change the percentages of 4's and 5's in the overall dataset to explore how dataset manipulation impacts a network.

This code block loads in the MNIST dataset.

In [22]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

This next code block will create an array of the 4's in the dataset and an array of the 5's, as well as corresponding arrays for the labels for these datasets. Then we will print the number of 4's and 5's to make sure there is an equal number.

In [23]:
four_train_x = []
four_train_y = []
five_train_x = []
five_train_y = []
num_train_four = 0
num_train_five = 0

for i in range(len(train_x)):
  if train_y[i] == 4 and num_train_four < 5000:
    four_train_x.append(train_x[i])
    four_train_y.append([1,0])
    num_train_four += 1
  elif train_y[i] == 5 and num_train_five < 5000:
    five_train_x.append(train_x[i])
    five_train_y.append([0,1])
    num_train_five += 1

print("number of fours: ", len(four_train_x))
print("number of fives: ", len(five_train_x))

number of fours:  5000
number of fives:  5000


Now, we must create arrays that contain only 4's and 5's to test the network. We additionally need this set to have an equal number of 4's and 5's, so we will print out the number of 4's and 5's to check that they are equal.

In [24]:
new_test_x = []
new_test_y = []
num_four = 0
num_five = 0

for i in range(len(test_x)):
  if test_y[i] == 4 and num_four < 750:
    new_test_x.append(test_x[i])
    new_test_y.append([1,0])
    num_four += 1
  elif test_y[i] == 5 and num_five < 750:
    new_test_x.append(test_x[i])
    new_test_y.append([0,1])
    num_five += 1

new_test_x = np.array(new_test_x)
new_test_y = np.array(new_test_y)

new_test_x_normalized = new_test_x/255

print("number of fours: ", num_four)
print("number of fives: ", num_five)

number of fours:  750
number of fives:  750


Here, we define a function that will allow us to split the data by percentages of 4's and 5's. This will then create an array that contains the entered percentages of 4's and 5's.

In [25]:
def split_data(pct4, pct5):
  assert pct4+pct5 == 1, "percentages must sum to 1"
  num4 = int(pct4*(len(four_train_x)))
  num5 = int(pct5*(len(five_train_x)))
  shuffle(four_train_x)
  shuffle(five_train_x)
  print("four train length:", num4)
  print("five train length:", num5)
  inputs = four_train_x[:num4] + five_train_x[:num5]
  targets = ([[1,0]]*num4) + ([[0,1]]*num5)
  mix = list(zip(inputs, targets))
  shuffle(mix)
  inputs, targets = zip(*mix)

  inputs = np.array(inputs)/255
  targets = np.array(targets)

  return inputs, targets


Now, **enter the percentage of 4's** you want in the dataset as a decimal. For example, if you want 25% 4's, enter 0.25. To start, split the data evenly. After seeing how the network looks with an even split of data, change the percentages and rerun all the code blocks below to see how the network changes.

In [26]:
pct4 = 0.5

**Enter the percentage of 5's** you want to be in the dataset. Make sure that the decimal percentage of 4's and 5's adds to one.

In [27]:
pct5 = 0.5

Run this code block to split up the data and put the composition of 4's and 5's into input and target arrays. Additionally, the number of 4's and 5's will be outputted; verify that the numbers look correct given the percentages of 4's and 5's you entered.

In [28]:
inputs, targets = split_data(pct4, pct5)

four train length: 2500
five train length: 2500


Now, we can see what the new dataset we will use looks like. Notice how the number of 4's and 5's varies based on the percentages you inputted.

In [29]:
images = [array_to_image(inputs[i]) for i in range(20)]
gallery(images)

0,1,2,3,4
5,6,7,8,9
10,11,12,13,14
15,16,17,18,19


Here we are creating the neural network. We will utilize a very simple network which first flattens the two-dimensional input, passes it through a single hidden layer, and then on to the output layer.

In [30]:
net = SimpleNetwork((28, 28), "Flatten", 25, (2, "softmax"))

Summarizing the network allows us to make sure it looks as it is expected.

In [31]:
net.summary()

Model: "SimpleNetwork"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 hidden_2 (Dense)            (None, 25)                19625     
                                                                 
 output (Dense)              (None, 2)                 52        
                                                                 
Total params: 19677 (76.86 KB)
Trainable params: 19677 (76.86 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


This is a fairly simple task for the network so it only needs 5 epochs of training to achieve high accuracy (each epoch is one pass through the training data).

In [32]:
history = net.fit(inputs,                                   # new training examples
                  targets,                                  # new training labels
                  verbose=1,                                # verbose output
                  validation_data=(new_test_x_normalized,   # validation examples
                                   new_test_y),             # validation labels
                  epochs=5)                                 # number of times to loop through the training set

Epoch 5/5 loss: 0.007666457910090685 - tolerance_accuracy: 0.9516321420669556 - val_loss: 0.00848546251654625 - val_tolerance_accuracy: 0.948803186416626


After training the network, **take note of the tolerance and value tolerance accuracy for every time you retrain the network with manipulated percentages**. The tolerance accuracy tells us how accurately the network has learned the data in the dataset it is trained on, where the value tolerance accuracy reports the accuracy of the network on the test dataset.

**val_tolerance_accuracy**: *enter here*

**tolerance_accuracy**: *enter here*

###Testing the Network

Now, we can look at inputs in the network to see if we get the expected output. For 4's, there should be a white block on the left and a black block on the right for the output. For 5's, there should be a black block on the left and a white block on the right for the output. For inputs that the network is having trouble recognizing, their output will not be clearly black or white in either of the two output blocks. Additionally, below the visualization of the network, when the test function is run, you can see percentages of certainty. The first number is the certainty that the digit is a four, and the second number is the certainty that the digit is a five.

In [33]:
from time import sleep
def test(net, n):
  for i in range(n):
    net.display(new_test_x_normalized[i])
    outputs = net.propagate(new_test_x_normalized[i])
    print(", ".join([str(round(v,2)) for v in outputs]))
    sleep(2)

We are looking at the first ten inputs for the test dataset. You may notice some strange looking examples. The third one is a 5, but with a much smaller bottom half. The ninth one appears to be a quickly drawn 4 where the bottom half is missing.

In [34]:
test(net, 10)

1.0, 0.0


As you can see from these examples, when the network's dataset is split evenly between 4's and 5's, it typically performs very well. It has an accuracy of around 95% on the test dataset (which it was not trained on) which is very significant. This shows that this network is very strong and effective at predicting whether a hand drawn digit is a 4 or a 5.

So far, we have created and shown the effectiveness of the neural network that is trained to assess whether a given digit in the test dataset is a 4 or 5. The network should perform with a high accuracy and confidence in its guesses for any given input digit. For some less clear examples, it may have a hard time distinguishing the digit, however, overall the network should perform with a good deal of accuracy when the dataset it is trained on has an equal number of 4's and 5's.

Now, we can see how many total errors this network had and which specific digits it classified incorrectly. Run this next code block to have a summary of the errors the network made, as well as the percentage of errors that were 4's and percentage of errors that were 5's. **Note how many errors the network made and which digit it classified incorrectly most often.**

In [37]:
from numpy import argmax
outputs = net.predict(new_test_x_normalized)
answers = [argmax(output) for output in outputs]
newtargets = [argmax(target) for target in new_test_y]
incorrect = [i for i in range(len(answers)) if answers[i] != newtargets[i]]
print("number of digits classified incorrectly:", len(incorrect))
missed_target = [targets[i] for i in incorrect]
wrong_answer = [answers[i] for i in incorrect]
per4 = 0
per5 = 0
for i in range(len(incorrect)):
  if wrong_answer[i] == 1:
    per4 += 1
  else:
    per5 += 1

print("\n")
print("percentage of errors on 4's:", per4/len(incorrect))
print("percentage of errors on 5's:", per5/len(incorrect))

number of digits classified incorrectly: 16


percentage of errors on 4's: 0.125
percentage of errors on 5's: 0.875


*write observations here*

Prints gallery of digits that were classified incorrectly.

In [36]:
images = [array_to_image(new_test_x[index]) for index in incorrect]
gallery(images)

0,1,2,3
4,5,6,7
8,9,10,11
12,13,14,15


After seeing which digits were classified incorrectly, consider **why** this may have occurred and **how** the percentage of 4's and 5's you inputted would have this effect.

*write observations here*

###Changing Dataset Composition

Now that we have shown how this network performs with an equal number of 4's and 5's in the dataset, we want to show how the performance and accuracy of the network changes when we manipulate the percentage of 4's and 5's in the dataset. **Return to where you entered the percentages of 4's and 5's in the training dataset and change the percentages, rerunning all the code blocks. To start, change the percentage of 4's to 25% and percentage of 5's to 75%.**

After rebuilding and retraining the network on a new percentage distribution of 4's and 5's, we must retest the network. **Go back to the testing the network section and note how the digits are classified, what percentage of 4's and 5's you used, and any errors you notice.**

*write observations here*

After you finish training your network with the first manipulated percentage of 4's and 5's, change the values. You can increase or decrease the percentages, retraining the network, and observe how well the network is able to recognize 4's and 5's in the test dataset. First, see if you can find a balance of 4's and 5's where the majority of errors occur for only one of the digits. Then, see if you can find a percentage of 5's where all the 5's start being categorized as 4's or vice versa.  **What are the tipping points?**

*write answer here*

Ensure that you are overrepresenting 4's as well as 5's as you go back to rerun this network on manipulated percentages. **How does the network's accuracy change when you overrepresent one digit versus when you overrepresent the other?** *Note which digit was overrepresented and what percentages you used in your observations.*

*write observations here*

After finishing testing the different percentages that you can use to show varying levels of efficacy in the network, consider the potential broader implications of dataset composition before moving on to the next section. **How could having a specific subsection of data that an AI is trained on being underrepresented have very real world consequences? What possible issues and biases can you see with human decision making that goes into creation of datasets that are used to train these networks?**

*write answer here*

## Implications of Dataset Composition
We have explored how manipulating a dataset can change a network's efficacy in recognition of specific numbers when they are under or overrepresented in a dataset. Datasets, and thus their composition, is an essential component of neural networks. Below, we will explore how bias in a dataset's composition can lead to negative impacts on marginalized communities.

Within the past couple of years, bias within algorithms and AI has begun to receive attention. Many computer scientists and researchers have begun to recognize inherent bias, known also as "algorithmic prejudices," present in algorithms, software, machine learning, artificial intelligence, and nearly every facet of computer science (see reference [3]). Within the context of datasets that are used for the training of neural networks, bias is pervasive. This bias becomes particularly concerning as algorithms are beginning to take over human responsibilities (see reference [2]). For example, algorithms are now being used by US law enforcement for "predictive justice"(see reference [3]). These tools "calculate the probability that a person will not show up for trial as scheduled or commit future crimes"(see reference [3]). As these algorithms become increasingly present in our society, we must evaluate and consider their inherent biases.

A major contributor to the current movement exploring and combatting biases in algorithms is Joy Buolamwini. As a graduate student at MIT, Buolamwini co-wrote the paper "Gender Shades: Intersectional Accuracy Disparities in Commercial Gender Classification" along with Timmit Gebru which explores the ways in which machine learning algorithms can discriminate based on classes like race and gender (see reference [1]). As Buolamwini describes in her Ted Talk, she was inspired to address bias in machine learning algorithms when as an undergraduate student at Georgia Tech a robot that was supposed to recognize faces could not detect her's, as a black woman. Among other findings, this paper revealed that while lighter-skinned males had an extremely low error rate of 0.8% while darker-skinned females had a significantly higher error rate of up to 34.7% (see reference [1]).

### Buolamwini's Work

In 2017, Buolamwini gave a Ted Talk demonstrating the discriminatory tendencies of widely used and accepted training sets and algorithms (see reference [2]). She refers to the concept of the "coded gaze" as algorithmic bias in the field of computer science. Within her talk, she dives deeper into the harms and discriminatory practices perpetrated by these training sets which are often severely lacking diversity. These practices include predictive policing. A study from the Georgetown Law Center showed that these police systems contain 1 in 2 adults in the US in a criminal facial recognition network (see reference [4]). These networks used by law enforcement have not been audited for accuracy and can result in misidentification of criminals, having a potentially serious consequence on the victim of this misidentification. With such serious stakes, it is essential to consider and address the biases of these algorithms and networks.

To see the Georgetown Law Center's full report on law enforcement's use of facial recognition and recommendations, please access this link: [Perpetual Line Up](https://www.perpetuallineup.org/).

Click here to watch Buolamwini's Ted Talk!

[![IMAGE ALT TEXT](http://img.youtube.com/vi/UG_X_7g63rY/0.jpg)](https://www.youtube.com/watch?v=UG_X_7g63rY "How I'm fighting bias in algorithms")

#### Gender Shades

"Gender Shades" tested 3 commercial gender classification systems (Microsoft, IBM, Face++) using a dataset specifically designed to determine the potential biases present in these systems (see reference [1]). The dataset (Pilot Parliaments Benchmark), specifically created for this study, was composed of faces of 1270 individuals from three African countries and three European countries. The individuals were each given skin type labels per the Fitzpatrick six-point labeling system and given gender labels, either female or male given the binary nature of the evaluation systems.

In evaluation of these classifiers, there were several main takeaways. Firstly, "male subjects were more accurately classified than female subjects"(reference [1] pg. 8). Additionally, lighter-skinned subjects were more accurately classified than those with darker skin. Further, all classifiers performed worst on darker female subjects (reference [1] pg. 8). Here is the complete summarized key findings as outlined in the study:

"All classifiers perform better on male faces
than female faces (8.1% − 20.6% difference
in error rate)

• All classifiers perform better on lighter faces
than darker faces (11.8% − 19.2% difference
in error rate)

• All classifiers perform worst on darker female
faces (20.8% − 34.7% error rate)

• Microsoft and IBM classifiers perform best
on lighter male faces (error rates of 0.0% and
0.3% respectively)

• Face++ classifiers perform best on darker
male faces (0.7% error rate)

• The maximum difference in error rate between the best and worst classified groups is
34.4%" (reference [1] pg. 8).

Further, this paper emphasizes the complete inability of these commercial systems to recognize gender minorities as they are completely excluded from datasets and classification options. Buolamwini notes, "The companies provide no documentation to clarify if their gender classification systems which provide sex labels are classifying gender identity or biological sex"(reference [1] pg. 6). As she emphasizes, "This reductionist view of gender does not adequately capture the complexities of gender or address trangender identities"(reference [1] pg. 6). When using these systems it is important to consider the erasure they create of people of non binary gender identities.

Buolamwini and Gerbru's study "Gender Shades" brought to the forefront the inherent biases present in well-established commercial classifiers and marginalization of those with intersectional identities, particularly darker skinned women, in these algorithms. The consequences of these prejudices have the potential to only further harm people of intersectional minority identities who are already marginalized in our society. As companies continue to develop these tools, Buolamwini calls for "inclusive benchmark datasets and subgroup accuracy reports" which will be "necessary to increase transparency and accountability in artificial intelligence"(reference [1] pg. 12). Continuing into the development of these tools, there will need to be increased "demographic and phenotypic transparency and accountability in artificial intelligence"(reference [1] pg. 12).

To have a more comprehensive understanding of Buolamwini and her co-collabrator Timmit Gebru's research "Gender Shades," you can read the full paper here:  [Gender Shades: Intersectional Accuracy Disparities in Commercial Gender Classification](https://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf).

###Reflect

After reading more about the biases present in machine learning algorithms, **how do you see your role as a member of a modern society in which the presence of these algorithms is only increasing? What are ways in which we can combat these biases?**

*write answer here*

Go back to the data manipulation section of this notebook and take note of the accuracy percentage you recorded when either fours or fives were overrepresented (particularly for when the minority digit represented 7% or less of the dataset). **Why is accuracy alone not always a reliable parameter? What are the fallacies underlying reporting the "accuracy" of a system? How could this number impact systems' usage and our trust in them?** Think about Buolamwini's findings. Contextualize your answer accordingly.

*write answer here*

## Navigating Biases
Considering ways in which we can work towards a more inclusive computing community.

As we work towards a more inclusive and less prejudiced computer science sphere, we must consider these issues, recognize them in our processes, and change our practices. A major component of changing the presence of these biases and their impact is focusing on inclusive coding practices. As Buolamwini outlines in her Ted Talk, we must consider who codes, how we code, and why we code (see reference [2]). Having a more diverse community of coders that consider and prioritize the needs and experiences of marginalized communities is an important step in creating a more inclusive field and algorithms.

Within the Georgetown Law Center report, the writers emphasize a need for significant legislative and regulatory change (see reference [4]). Law enforcement's use of facial recognition has the potential to do real damage, if it has not already impacted countless individuals. The report suggests legislation should be passed to regulate these technologies including requiring reasonable suspicion to use facial recognition, only use mug shot databases, court approval to use ID photos and license photos, requiring probable cause to use surveillance footage, completely ban tracking individuals for free speech issues, and increase accuracy testing. Further, they suggest a complete reform to the FBI facial recognition systems. They argue that these systems must be transparent and held publicly accountable, releasing statistics relating to arrest numbers. Importantly, they call for testing of racial bias within these systems and datasets that reflect the diversity of the American population. All of these reforms are important to implement if we want to mitigate the potential harm that these law enforcement agencies can perpetuate against already vulnerable communities.

As Buolamwini emphasizes at the end of her Ted Talk, we must create "a world where technology works for all of us, not just some of us, a world where we value inclusion and center social change." To finish her talk, she poses a question: "Will you join me in the fight?" **After reading through this computational essay, consider why it is important to join this "fight"? What are your personal motivations behind creating a more inclusive computing space and why is it important?**

*write answer here*

## References


[1] Buolamwini, Joy and Timnit Gebru. Gender Shades: Intersectional Accuracy Disparities in Commercial Gender Classification.  Conference on Fairness, Accountability, and Transparency, 2018. https://proceedings.mlr.press/v81/buolamwini18a/buolamwini18a.pdf.

[2] Buolamwini, Joy. How I’m Fighting Bias in Algorithms. 1489075733. www.ted.com, https://www.ted.com/talks/joy_buolamwini_how_i_m_fighting_bias_in_algorithms.

[3] Cataleta, Maria Stefania. Humane Artificial Intelligence: The Fragility of Human Rights Facing AI. East-West Center, 2020. JSTOR, https://www.jstor.org/stable/resrep25514.

[4] “The Perpetual Line-Up.” Perpetual Line Up, https://www.perpetuallineup.org/. Accessed 10 June 2024.